<h1><center>F1 RACE WINNER PREDICTOR</center></h1>
<h1><center>2. Data Preprocessing</center></h1>

---

## In this Notebook:

In this Notebook you will find the preliminary cleaning of the data and a quick feature engineering related to the use of dates and qualifying times. 

1. [Our raw data](#1.-Raw-data) 
2. [Dropping unnecessary columns and merging](#2.-Dropping-unnecessary-columns-and-merging)
3. [Data cleaning](#3.-Data-cleaning)
4. [Preliminary feature engineering](#4.-Feature-engineering-(preliminary))  
    4.1. [Drivers' age](#4.1.-Drivers'-age)  
    4.2. [Quali delta time](#4.2.-Qualifying-time-delta)

The following steps after preprocessing are 3. Exploratory Data Analysis and 4. Modeling.

---

In [6]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. Raw data

In [2]:
races = pd.read_csv('data/races.csv')
results = pd.read_csv('data/results.csv')
qualifying = pd.read_csv('data/qualifying.csv')
driver_standings = pd.read_csv('data/driver_standings.csv')
constructor_standings = pd.read_csv('data/constructor_standings.csv')
weather = pd.read_csv('data/weather.csv')

In [3]:
races.tail(3)

,season,round,circuit_id,lat,long,country,date,url
1054,2021,20,losail,25.4900,51.4542,Qatar,2021-11-21,http://en.wikipedia.org/wiki/2021_Qatar_Grand_...
1055,2021,21,jeddah,21.6319,39.1044,Saudi Arabia,2021-12-05,http://en.wikipedia.org/wiki/2021_Saudi_Arabia...
1056,2021,22,yas_marina,24.4672,54.6031,UAE,2021-12-12,http://en.wikipedia.org/wiki/2021_Abu_Dhabi_Gr...


In [4]:
results.tail(3)

,season,round,circuit_id,driver,date_of_birth,nationality,constructor,grid,time,status,points,podium,url
24944,2021,22,yas_marina,russell,1998-02-15,British,williams,17,NaN,Gearbox,0.0,18,http://en.wikipedia.org/wiki/2021_Abu_Dhabi_Gr...
24945,2021,22,yas_marina,raikkonen,1979-10-17,Finnish,alfa,18,NaN,Brakes,0.0,19,http://en.wikipedia.org/wiki/2021_Abu_Dhabi_Gr...
24946,2021,22,yas_marina,mazepin,1999-03-02,Russian,haas,20,NaN,Illness,0.0,20,http://en.wikipedia.org/wiki/2021_Abu_Dhabi_Gr...


In [5]:
qualifying.tail(3)

,grid_position,driver_name,car,qualifying_time,season,round
15330,17,George Russell RUS,Williams Mercedes,1:24.423,2021,22
15331,18,Kimi Räikkönen RAI,Alfa Romeo Racing Ferrari,1:24.779,2021,22
15332,19,Mick Schumacher MSC,Haas Ferrari,1:24.906,2021,22


In [6]:
# quali.grid_position must be the same as results.grid

qualifying.rename(columns = {'grid_position': 'grid'}, inplace = True)

In [7]:
driver_standings.tail(3)

,season,round,driver,driver_points_after_race,driver_wins_after_race,driver_standings_pos_after_race,driver_points,driver_wins,driver_standings_pos
27110,2021,22,mick_schumacher,0.0,0,19,0.0,0.0,19.0
27111,2021,22,kubica,0.0,0,20,0.0,0.0,20.0
27112,2021,22,mazepin,0.0,0,21,0.0,0.0,21.0


In [8]:
constructor_standings.tail(3)

,season,round,constructor,constructor_points_after_race,constructor_wins_after_race,constructor_standings_pos_after_race,constructor_points,constructor_wins,constructor_standings_pos
12708,2021,22,williams,23.0,0,8,23.0,0.0,8.0
12709,2021,22,alfa,13.0,0,9,13.0,0.0,9.0
12710,2021,22,haas,0.0,0,10,0.0,0.0,10.0


In [9]:
weather.tail(3)

,season,round,circuit_id,weather,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy
1054,2021,20,losail,Clear,1,0,0,0,0
1055,2021,21,jeddah,Clear,1,0,0,0,0
1056,2021,22,yas_marina,Clear,1,0,0,0,0


# 2. Dropping redundant columns and merging

In [10]:
driver_standings.drop(['driver_points_after_race', 'driver_wins_after_race', 'driver_standings_pos_after_race'] ,axis = 1, inplace = True)
driver_standings.tail(3)

,season,round,driver,driver_points,driver_wins,driver_standings_pos
27110,2021,22,mick_schumacher,0.0,0.0,19.0
27111,2021,22,kubica,0.0,0.0,20.0
27112,2021,22,mazepin,0.0,0.0,21.0


In [11]:
constructor_standings.drop(['constructor_points_after_race', 'constructor_wins_after_race','constructor_standings_pos_after_race' ],axis = 1, inplace = True)
constructor_standings.tail(3)

,season,round,constructor,constructor_points,constructor_wins,constructor_standings_pos
12708,2021,22,williams,23.0,0.0,8.0
12709,2021,22,alfa,13.0,0.0,9.0
12710,2021,22,haas,0.0,0.0,10.0


In [12]:
# Merging
df_1 = pd.merge(races, weather, how='inner', on=['season', 'round', 'circuit_id']).drop(['lat', 'long','country'], axis = 1)
df_2 = pd.merge(df_1, results, how='inner', on=['season', 'round', 'circuit_id', 'url']).drop(['url','points', 'status', 'time'], axis = 1)
df_3 = pd.merge(df_2, driver_standings, how='left', on=['season', 'round', 'driver']) 
df_4 = pd.merge(df_3, constructor_standings, how='left', on=['season', 'round', 'constructor']) #from 1958

final_df = pd.merge(df_4, qualifying, how='inner', on=['season', 'round', 'grid']).drop(['driver_name', 'car'], axis = 1) # from 1983 onwards

In [13]:
final_df.tail(3)

,season,round,circuit_id,date,weather,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,...,constructor,grid,podium,driver_points,driver_wins,driver_standings_pos,constructor_points,constructor_wins,constructor_standings_pos,qualifying_time
15295,2021,22,yas_marina,2021-12-12,Clear,1,0,0,0,0,...,alfa,14,17,3.0,0.0,18.0,13.0,0.0,9.0,1:24.251
15296,2021,22,yas_marina,2021-12-12,Clear,1,0,0,0,0,...,williams,17,18,16.0,0.0,15.0,23.0,0.0,8.0,1:24.423
15297,2021,22,yas_marina,2021-12-12,Clear,1,0,0,0,0,...,alfa,18,19,10.0,0.0,16.0,13.0,0.0,9.0,1:24.779


In [14]:
final_df.columns

Index(['season', 'round', 'circuit_id', 'date', 'weather', 'weather_warm',
       'weather_cold', 'weather_dry', 'weather_wet', 'weather_cloudy',
       'driver', 'date_of_birth', 'nationality', 'constructor', 'grid',
       'podium', 'driver_points', 'driver_wins', 'driver_standings_pos',
       'constructor_points', 'constructor_wins', 'constructor_standings_pos',
       'qualifying_time'],
      dtype='object')

In [15]:
final_df.shape

(15298, 23)

# 3. Data cleaning

In [16]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15298 entries, 0 to 15297
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   season                     15298 non-null  int64  
 1   round                      15298 non-null  int64  
 2   circuit_id                 15298 non-null  object 
 3   date                       15298 non-null  object 
 4   weather                    15298 non-null  object 
 5   weather_warm               15298 non-null  int64  
 6   weather_cold               15298 non-null  int64  
 7   weather_dry                15298 non-null  int64  
 8   weather_wet                15298 non-null  int64  
 9   weather_cloudy             15298 non-null  int64  
 10  driver                     15298 non-null  object 
 11  date_of_birth              15298 non-null  object 
 12  nationality                15298 non-null  object 
 13  constructor                15298 non-null  obj

In [17]:
final_df.isna().sum()/len(final_df) # what PERCENTAGE of our values are null? Very few, we can drop

season                       0.000000
round                        0.000000
circuit_id                   0.000000
date                         0.000000
weather                      0.000000
weather_warm                 0.000000
weather_cold                 0.000000
weather_dry                  0.000000
weather_wet                  0.000000
weather_cloudy               0.000000
driver                       0.000000
date_of_birth                0.000000
nationality                  0.000000
constructor                  0.000000
grid                         0.000000
podium                       0.000000
driver_points                0.036018
driver_wins                  0.036018
driver_standings_pos         0.036018
constructor_points           0.010197
constructor_wins             0.010197
constructor_standings_pos    0.010197
qualifying_time              0.021898
dtype: float64

**Note** for 'driver_points', 'driver_wins', 'driver_standings_pos', 'constructor_points','constructor_wins' and 'constructor_standings_pos', we can replace the null values by zero. We cannot do the same with qualifying_time, since a qualifying time of zero corresponds to an infinitely fast car... In this case we replace them with the average qualifying time of that team for that circuit.

In [18]:
# fill/drop nulls. Not for quali_times, those will be handeled later
for col in ['driver_points', 'driver_wins', 'driver_standings_pos', 'constructor_points', 'constructor_wins' , 'constructor_standings_pos']:
    final_df[col].fillna(0, inplace = True)
    final_df[col] = final_df[col].map(lambda x: int(x))

# 4. Feature engineering (preliminary)

## 4.1. Drivers' age

In [19]:
# having current dates and birth dates by themselves is useless so we calculate the drivers' ages with them

from dateutil.relativedelta import *

final_df['date'] = pd.to_datetime(final_df.date) # change data type to date
final_df['date_of_birth'] = pd.to_datetime(final_df.date_of_birth) # change data type to date
final_df['driver_age'] = final_df.apply(lambda x: relativedelta(x['date'], x['date_of_birth']).years, axis=1) # calc age
final_df.drop(['date', 'date_of_birth'], axis = 1, inplace = True) # drop

In [20]:
final_df.shape

(15298, 22)

## 4.2. Qualifying time delta

In [21]:
# check for null values in season 2021
final_df_2021 = final_df[final_df['season'] == 2021]
final_df_2021.isna().sum()

season                        0
round                         0
circuit_id                    0
weather                       0
weather_warm                  0
weather_cold                  0
weather_dry                   0
weather_wet                   0
weather_cloudy                0
driver                        0
nationality                   0
constructor                   0
grid                          0
podium                        0
driver_points                 0
driver_wins                   0
driver_standings_pos          0
constructor_points            0
constructor_wins              0
constructor_standings_pos     0
qualifying_time              70
driver_age                    0
dtype: int64

In [22]:
# remove the minutes from qualifying times
final_df['qualifying_time'] = final_df['qualifying_time'].apply(lambda x: float(x.split(':')[1]) if (len(str(x)) >= 8) else x)

# convert all to float in only seconds
final_df['qualifying_time'] = final_df['qualifying_time'].astype(float)

# remove the 2020 Bahrain Grand Prix, as values are corrupted
final_df = final_df.drop(final_df[(final_df['circuit_id'] == 'bahrain') & (final_df['season'] == 2020)].index)

# fill quali nan's with the average time of the team in that circuit and that season. 
# cascading imputation
final_df['qualifying_time'] = final_df['qualifying_time'].fillna(final_df.groupby(['circuit_id', 'constructor', 'season'])['qualifying_time'].transform('mean'))
final_df['qualifying_time'] = final_df['qualifying_time'].fillna(final_df.groupby(['circuit_id', 'constructor'])['qualifying_time'].transform('mean'))
final_df['qualifying_time'] = final_df['qualifying_time'].fillna(final_df.groupby(['circuit_id'])['qualifying_time'].transform('mean'))

# calculate delta in qualifying times
final_df = final_df[final_df['qualifying_time'] != 0] 
final_df.sort_values(['season', 'round', 'grid'], inplace = True)
final_df['qualifying_time_diff'] = final_df.groupby(['season', 'round']).qualifying_time.diff()
final_df['qualifying_time'] = final_df.groupby(['season', 'round']).qualifying_time_diff.cumsum().fillna(0)
final_df.drop('qualifying_time_diff', axis = 1, inplace = True)

In [40]:
# check if we have all races for 2021 season
have=list(final_df_2021.circuit_id.unique()) # 22 races
musthave=list(races[races['season'] == 2021].circuit_id.reset_index(drop=True).unique()) # 22 races
have==musthave

In [25]:
# get dummies

df_dum = pd.get_dummies(final_df, columns = ['circuit_id', 'nationality', 'constructor'] )

# drop dummy columns that do not have enough 1's

for col in df_dum.columns:
    if 'nationality' in col and df_dum[col].sum() < 140:
        df_dum.drop(col, axis = 1, inplace = True)
        
    elif 'constructor' in col and df_dum[col].sum() < 140:
        df_dum.drop(col, axis = 1, inplace = True)
        
    elif 'circuit_id' in col and df_dum[col].sum() < 70:
        df_dum.drop(col, axis = 1, inplace = True)
    
    else:
        pass

In [26]:
df_dum.shape

(15254, 103)

In [27]:
final_df.shape

(15254, 22)

In [29]:
final_df.to_csv('data/df_no_dummies.csv', index=False)

## Next step: EDA